# **Práctica 2**
* **Alumno 1**: Bolinches Segovia, Jorge
* **Alumno 2**: Cerezo Pomykol, Jan
***

### **Índice**
* [Carga de datasets](#1)
* [Entrenamiento y evaluación del Perceptrón](#2)
* [Ensemble del clasificador](#3)

### **Carga de datasets** <a class="anchor" id="1"></a>

In [2]:
import numpy as np
import pandas as pd
import os.path
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, fetch_openml

# Iris
dataset = load_iris()
X_iris = dataset.data
y_iris = dataset.target

# Wine
dataset = load_wine()
X_wine = dataset.data
y_wine = dataset.target

# Cancer
dataset = load_breast_cancer()
X_cancer = dataset.data
y_cancer = dataset.target


# Isolet
# Si existe la base de datos, cargo las variables
if os.path.exists("isolet_X.pickle"):
    X = pd.read_pickle('isolet_X.pickle')
    y = pd.read_pickle('isolet_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("isolet_X.pickle")
    y.to_pickle("isolet_y.pickle")

X_isolet = np.array(X)
y_isolet = pd.factorize(y)[0]

# MNIST
# Si existe la base de datos, cargo las variables
if os.path.exists("mnist_X.pickle"):
    X = pd.read_pickle('mnist_X.pickle')
    y = pd.read_pickle('mnist_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("mnist_X.pickle")
    y.to_pickle("mnist_y.pickle")

X_mnist = np.array(X)
y_mnist = pd.factorize(y)[0]

datasets = {"iris": (X_iris, y_iris),
            "wine": (X_wine, y_wine),
            "cancer": (X_cancer, y_cancer),
            "isolet": (X_isolet, y_isolet),
            "mnist": (X_mnist, y_mnist)}

In [3]:
print("Dimensiones de los datasets:")
for i in datasets:
    print(i, ":\t", datasets[i][0].shape, sep='')

Dimensiones de los datasets:
iris:	(150, 4)
wine:	(178, 13)
cancer:	(569, 30)
isolet:	(7797, 617)
mnist:	(70000, 784)


## **Entrenamiento y evaluación del Perceptrón** <a class="anchor" id="2"></a>

In [4]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from time import time

Para la evaluación del Perceptrón vamos a emplear la siguiente función, que recibe como parámetros el dataset (X, y) y la lista de valores de fracción de validación. Devuelve como resultado un array con las tasas de acierto para cada valor de esta lista, así como el índice del mejor valor.

In [4]:
def find_val(X, y, v):
    nv = len(v)
    scores = np.empty((nv, 10))
    scores_mean = np.empty(nv)
    best_v = 0
    prev = 0
    for vi in range(nv):
        clf = Perceptron(early_stopping=True, random_state=0, n_jobs=-1, validation_fraction=v[vi])
        kf = KFold(n_splits=10)
        n_iter = 1
        inicial = True
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            if inicial:
                clf.set_params(eta0=1)
                inicial = False
            else:
                clf.set_params(eta0=1/clf.n_iter_)
            scores[vi][n_iter-1] = clf.fit(X_train, y_train).score(X_test, y_test)
            n_iter = n_iter + 1
        scores_mean[vi] = np.mean(scores[vi])
        if scores_mean[vi] > prev:
            best_v = vi
            prev = scores_mean[vi]
    return scores_mean, best_v

## Dataset iris

In [5]:
t0 = time()
interval = 0.1
val = np.arange(0.02, 0.4, 0.005)
scores, best_v = find_val(X_iris, y_iris, val)
print("iris: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

iris: max_score: 0.6667, std: 0.0843, best_val: 0.0250, time: 6.1336s


Para el dataset de Iris, la mejor proporción de validación interna que hemos encontrado es 0.025 (2.5%), dando una tasa de aciertos del 66.67% (33.33% de fallos). Esta tasa tan baja se debe a que el dataset consta de 3 clases, y dos de ellas no son linealmente separables. Se asignan datos correctamente a una clase, pero los datos correspondientes a las otras dos se mezclan.

## Dataset wine

In [6]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.005)
scores, best_v = find_val(X_wine, y_wine, val)
print("wine: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

wine: max_score: 0.8598, std: 0.1814, best_val: 0.4350, time: 14.0112s


La mejor fracción de validación interna que hemos encontrado para el dataset Wine es 0.4350, dando una tasa de aciertos del 85.98% (14% de fallos).

## Dataset cancer

In [7]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.001)
scores, best_v = find_val(X_cancer, y_cancer, val)
print("cancer: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

cancer: max_score: 0.8947, std: 0.0666, best_val: 0.7330, time: 20.8467s


En cuanto al dataset Cancer, la mejor fracción de validación interna que hemos encontrado es 0.733, dando una tasa de aciertos del 89.47% (10% de fallos).

## Dataset isolet

In [8]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.01)
scores, best_v = find_val(X_isolet, y_isolet, val)
print("isolet: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

isolet: max_score: 0.9277, std: 0.0217, best_val: 0.1300, time: 385.9607s


Los mejores resultados con el Perceptrón que hemos obtenido han sido con el dataset de Isolet. La fracción de validación interna seleccionada es 0.13, dando una tasa de aciertos del 92.77% (7.2% de fallos).

## Dataset MNIST

In [9]:
t0 = time()
val = np.arange(0.05, 1, 0.05)
scores, best_v = find_val(X_mnist, y_mnist, val)
print("mnist: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

mnist: max_score: 0.8688, std: 0.0081, best_val: 0.7000, time: 465.0147s


La mejor fracción de validación interna que hemos encontrado para el dataset MNIST es 0.7. Como se puede observar, ha tardado unos 490 segundos en buscar el mejor parámetro de entre una lista de 19 (val). No vamos a buscar uno mejor dado que tardaría demasiado. Con este parámetro, hemos conseguido una tasa de aciertos de 86.88% (13% de aciertos).

***
#### **Resumen de resultados**

**Nota**: no hemos buscado mejores valores, dado que tarda demasiado tiempo en ejecutar la función en algunos casos.

La siguiente tabla muestra la mejor tasa de aciertos que hemos conseguido con el Perceptrón para cada dataset, así como el valor de la fracción de validación interna seleccionado.

|dataset|fraccion de validación interna|tasa de aciertos|
|---|---|---|
|iris|0.0250|0.6667|
|wine|0.4350|0.8598|
|cancer|0.7330|0.8947|
|isolet|0.1300|0.9277|
|mnist|0.7000|0.8688|


In [10]:
def find_best_parameters_perceptron(X, y, eta0, vals):
    perceptron = Perceptron()
    params = {'eta0': eta0, 'validation_fraction': vals}
    gscv = GridSearchCV(perceptron, params, n_jobs=-1, scoring='accuracy', cv=10).fit(X, y)
    best_clf = gscv.best_estimator_
    result_score_mean = gscv.cv_results_['mean_test_score'][gscv.best_index_]
    result_score_std = gscv.cv_results_['std_test_score'][gscv.best_index_]
    print("eta0: %.4f, validation_fraction: %.4f, score: %.4f, std: %.4f" % (best_clf.eta0, best_clf.validation_fraction, result_score_mean, result_score_std))
    return best_clf

## **Ensemble del clasificador** <a class="anchor" id="3"></a>

In [29]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

def ensembler(X, y, name):
    X_aux = np.copy(X)
    y_aux = np.copy(y)
    estimators = []
    i = 0
    anterior = 0

    #mientras haya fallos o sea la primera iteración

    while(len(np.unique(y_aux))>1 or i==0):

        #se crea un perceptron
        clf = Perceptron(eta0=0.5, early_stopping=True, random_state=0)

        #se entrena con el dataset de errores de la iteración
        clf = clf.fit(X_aux, y_aux)


        #se añade a la lista de clasificadores
        estimators.append((str(i), clf))

        #se crea el ensemble con la lista de clasificadores
        Vclf = VotingClassifier(estimators=estimators, voting = 'hard', n_jobs= -1)

        #se entrena el ensemble
        Vclf = Vclf.fit(X, y)

        #se comprueba los aciertos del ensemble
        predicts = Vclf.predict(X)
        scores = cross_val_score(Vclf, X, y, scoring='accuracy', cv=10)
        tasa = scores.mean()



        #Se extraen los datos mal clasificados por el ensemble y sus correspondientes etiquetas
        X_aux = X[y!=predicts]
        y_aux = y[y!=predicts]

        #Se comprueba que la tasa de acierto haya mejorado con respecto a la anterior iteración
        # que la tasa de acierto sea mayor que el 95%
        # y que el numero de clases en las que ha fallado sea mayor que 1 como condiciones de parada

        v, c = np.unique(y_aux, return_counts=True)
        if(tasa<=anterior or tasa>0.95 or np.sum(c<2)):
            break

        #se incrementa la iteración y se actualiza la tasa de acierto para la siguiente iteración
        i+=+1
        anterior = tasa
    print(name + ":\tscore: %0.4f (std: %0.4f)" % (tasa, scores.std()))

In [22]:
t0 = time()
ensembler(X_iris, y_iris, 'iris')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_wine,y_wine, 'wine')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_cancer, y_cancer, 'cancer')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_isolet, y_isolet, 'isolet')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_mnist, y_mnist, 'mnist')
print("\ttime: %.4fs" % (time()-t0))

iris:	score: 0.6667 (std: 0.0000)
	time: 0.0840s
wine:	score: 0.5297 (std: 0.1177)
	time: 0.1730s
cancer:	score: 0.7823 (std: 0.2063)
	time: 0.1120s
isolet:	score: 0.9259 (std: 0.0147)
	time: 21.1489s
mnist:	score: 0.8687 (std: 0.0144)
	time: 185.5608s


## Dataset iris

In [30]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.0250, n_jobs=-1)
t0 = time()
#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_iris, y_iris)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_iris, y_iris, scoring='accuracy', cv=10)

print("iris:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Iris'))

iris:	score: 0.8133, std: 0.1293, time: 0.4371s


## Dataset wine

In [31]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.4350, n_jobs=-1)
t0 = time()
#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_wine, y_wine)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_wine, y_wine, scoring='accuracy', cv=10)

print("wine:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Wine'))

wine:	score: 0.6304, std: 0.0753, time: 0.4231s


## Dataset cancer

In [32]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.7330, n_jobs=-1)
t0 = time()
#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_cancer, y_cancer)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_cancer, y_cancer, scoring='accuracy', cv=10)

print("cancer:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Cancer'))

cancer:	score: 0.8823, std: 0.0422, time: 0.1040s


## Dataset isolet

In [33]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.1300, n_jobs=-1)
t0 = time()
#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_isolet, y_isolet)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_isolet, y_isolet, scoring='accuracy', cv=10)

print("isolet:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Isolet'))

isolet:	score: 0.9256, std: 0.0153, time: 29.6337s


## Dataset mnist

In [34]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.7000, n_jobs=-1)
t0 = time()
#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_mnist, y_mnist)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_mnist, y_mnist, scoring='accuracy', cv=10)

print("mnist:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Mnist'))

mnist:	score: 0.8796, std: 0.0080, time: 136.8499s
